# 7.15. General Depth First Search
The knight’s tour is a special case of a depth first search where the goal is to create the deepest depth first tree, without any branches. The more general depth first search is actually easier. Its goal is to search as deeply as possible, connecting as many nodes in the graph as possible and branching where necessary.


It is even possible that a depth first search will create more than one tree. When the depth first search algorithm creates a group of trees we call this a depth first forest. As with the breadth first search our depth first search makes use of predecessor links to construct the tree. In addition, the depth first search will make use of two additional instance variables in the Vertex class. The new instance variables are the discovery and finish times. The discovery time tracks the number of steps in the algorithm before a vertex is first encountered. The finish time is the number of steps in the algorithm before a vertex is colored black. As we will see after looking at the algorithm, the discovery and finish times of the nodes provide some interesting properties we can use in later algorithms.

## DFS 주요 특징

- 자유롭게 node 간 연결 가능. 복수의 tree 가능 ("depth first forest")


- 갈림길이 나타날 때마다 '다른 길이 있다'는 정보만 기록하면서**("grey")** 자신이 지나간 길을 지워나간다. 그러다 막다른 곳에 도달하면 직전 갈림길까지 돌아가면서 '이 길은 아님'이라는 표식**("black")**을 남긴다. 그렇게 갈림길을 순차적으로 탐색해 나가다 목적지를 발견하면 그대로 해답을 내고 종료.


- 자동 미로 생성에 많이 사용되는 알고리즘이기도 하다. 
 방향은 무작위로 해서 계속 뚫다가 막혀서 못 뚫으면 뚫을 수 있는 곳이 발견될 때까지 되돌아가서 다시 뚫고, 
  또 막히면 되돌아가고 이런 식으로 무한히 반복하다 보면 생긴다. 
  게다가 이 방식으로 미로를 만들면 빠져나가는 경로 또한 단 하나만 생긴다. 직접 보고 싶다면 이 동영상을 보자.
 https://youtu.be/0kaHIfrB3T4
 

In [9]:
# Vertex Class

class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])

    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id

    def getWeight(self,nbr):
        return self.connectedTo[nbr]
    
# Graph Class

class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,cost=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())

The code for our depth first search is shown in Listing 5. Since the two functions dfs and its helper dfsvisit use a variable to keep track of the time across calls to dfsvisit we chose to implement the code as methods of a class that inherits from the Graph class. This implementation extends the graph class by adding a time instance variable and the two methods dfs and dfsvisit. Looking at line 11 you will notice that the dfs method iterates over all of the vertices in the graph calling dfsvisit on the nodes that are white. The reason we iterate over all the nodes, rather than simply searching from a chosen starting node, is to make sure that all nodes in the graph are considered and that no vertices are left out of the depth first forest. It may look unusual to see the statement for aVertex in self, but remember that in this case self is an instance of the DFSGraph class, and iterating over all the vertices in an instance of a graph is a natural thing to do.

In [ ]:
from pythonds.graphs import Graph

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()    #super()???
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')       #모든 vertex에 대해서 초기화 
            aVertex.setPred(-1)             # 색상: white, 이전연결: -1
            
            
        for aVertex in self:                # white인 모든 vertex 에 대해서  dfsvisit ==> vertex에 대해서 시작점으로 점검함
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')        # 시작vertex 색상 white
        self.time += 1                      # 탐색 횟수 +1
        startVertex.setDiscovery(self.time) # 시작 vertex의 Discoverytime(첫 탐색시까지의 소요 step)
        
        for nextVertex in startVertex.getConnections():    #시작 vertex에서 연결 가능한 next vertex에 대해
            if nextVertex.getColor() == 'white':           # 미 탐색 vertex의  setPred로 연결함
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)                  # next vertex 기준. 재귀  ==> 깊이 있게 끝까지 가봄(Depth First Search)
                
        startVertex.setColor('black')                       # 더이상 연결할 vertex가 없다면 black으로, 탐색횟수 +1
        self.time += 1
        startVertex.setFinish(self.time)

<img src='https://upload.wikimedia.org/wikipedia/commons/2/2c/Depthfirst.png', width=200, height=300>

### Color
 - grey : 탐색중 
 - white: 미탐색  
 - black: 탐색완료(연결가능한 vertex가 남아있지 않음)

### step
 - starting time: vertex에 처음으로 도달하기 전 필요한 step 수
 - finishing time : vertex가 black으로 칠해지기전 필요한 step 수


<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsa.png'>

<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsb.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsc.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsd.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfse.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsf.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsg.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsh.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsi.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsj.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsk.png'>
<img src='http://interactivepython.org/runestone/static/pythonds/_images/gendfsl.png'>

The starting and finishing times for each node display a property called the parenthesis property. This property means that all the children of a particular node in the depth first tree have a later discovery time and an earlier finish time than their parent. Figure 26 shows the tree constructed by the depth first search algorithm.

##  DFS's Big(O) :  O(V+E)   
 * V: Vertex 개수(초기화 진행시)
 * E: Edge 개수(모든 Edge 에 대해서 탐색)

# 7.17 Topological Sorting

## Process 상에서 어떤 순서로 진행해야 하는지..  ex) 팬케잌 
### Critical Path Point 와 유사할 듯?¶

The topological sort is a simple but useful adaptation of a depth first search. The algorithm for the topological sort is as follows:

1. 시작지점을 선택하여 dfs 그래프 그린다: vertex별 finish time 확인 필요함. 
   (시작지점 선택시 incomming edge 가 없어야 함)
2. sorting: finish time의 역순
3. 리스트 생성 (1차원으로 표시)
※ directed acyclic graph에서만 가능: Cycle이 없어야만 함. / edge (v,w) : v 이후에 w 진행. v--> w

<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/pancakesDFS.png'>

<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/pancakesTS.png'>

# 참고) DFS Vs Topological Sorting



<img src = 'http://www.geeksforgeeks.org/wp-content/uploads/graph.png', width=300, height=400>

 - 5 - 2 - 3 - 1 - 0 - 4   : DFS (O)   Vs  Topological Sorting(X)
 
 - 5 - 4 - 2 - 3 - 1 - 0   : DFS (X)   Vs  Topological Sorting(O)
 
 - 4 - 5 - 2 - 3 - 1 - 0   : DFS (X)   Vs  Topological Sorting(O)


*source: http://www.geeksforgeeks.org/topological-sorting/

# 7.18. Strongly Connected Components

<img src = ' http://interactivepython.org/runestone/static/pythonds/_images/scc1.png'>
<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/scc2.png'>

## strongly connected components algorithm (SCC) 만드는 법


### Kosaraju's algorithm

 1. graph G의 dfs 계산 --> finish time 필요함
 
 
 2. GT 구함 : G에서 Edge의 방향이 반대.   
      ex)G :  1 --> 2 --> 3                 GT:   1 <-- 2 <-- 3
      ex)             --> 4                               <-- 4
      
      
 3. GT 의 dfs 계산( 1번에서 계산한 finish time 의 역순으로 진행)
 
 
 4. 3번에서 도출된 트리는  strongly connected components 임. 

### 예제

1. Graph G 
<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/scc1a.png'>

2-3. GT 
<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/scc1b.png'>

/4. strongly connected components
<img src = 'http://interactivepython.org/runestone/static/pythonds/_images/sccforest.png'>